In [3]:
from itertools import combinations
import numpy as np
import icecream as ic

## Simple Test Problem

In [7]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('./lab2/test_problem.npy')

## Common tests

In [4]:
problem = np.load('lab2/problem_r2_100.npy')

In [5]:
# Negative values?
np.any(problem < 0)

np.True_

In [6]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [7]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [8]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False

In [9]:
def cost(adj, sol):

    s = np.asarray(sol, dtype=int)
    to_idx = np.roll(s, -1)
    return float(np.sum(adj[s, to_idx]))

In [10]:
def starting_point(problem):
    n = problem.shape[0]
    rng = np.random.default_rng()
    return rng.permutation(np.arange(0, n))

In [ ]:
def swap_mutation(solution, mutation_rate=0.05):
    """
    swap mutation with probability: mutation_rate
    """
    new_solution = solution.copy()
    rng = np.random.default_rng()
    
    if rng.random() < mutation_rate:
        n = len(new_solution)
        idx1, idx2 = rng.integers(0, n, 2)
        while idx1 == idx2:
            idx2 = rng.integers(0, n)
            
        new_solution[idx1], new_solution[idx2] = new_solution[idx2], new_solution[idx1]
        
    return new_solution

In [ ]:
def order_crossover_ox1(parent1, parent2):
    """
    Crossover OX1
    mantain certain parent1 gene and fill the splution with the remanent parent2 gene in order
    """
    rng = np.random.default_rng()
    n = len(parent1)
    
    # 1. Choose two cut points
    cut1, cut2 = rng.integers(0, n, 2)
    if cut1 > cut2:
        cut1, cut2 = cut2, cut1
    if cut1 == cut2:
        cut2 += 1 # minimun 1 elemente
        
    child = -np.ones(n, dtype=int)
    
    # 2.copy of parent1 genes
    child[cut1:cut2] = parent1[cut1:cut2]
    
    # 3. vector for parent2 genes
    parent2_genes = []
    
    genes_in_child = set(child[cut1:cut2])
    for gene in parent2:
        if gene not in genes_in_child:
            parent2_genes.append(gene)
            
    # 4. fill child with parent2 gene only if parent1 gene is not present
    child_idx = cut2
    parent2_idx = 0
    
    while parent2_idx < len(parent2_genes):
        if child_idx == n:
            child_idx = 0
            
        if child_idx == cut1:
            child_idx = cut2
        
        child[child_idx] = parent2_genes[parent2_idx]
        child_idx += 1
        parent2_idx += 1
        
    return child

In [ ]:
def tournament_selection(population, costs, k=3):
    """
    choose random k element in population and return the best one
    """
    rng = np.random.default_rng()
    
    indices = rng.choice(len(population), k, replace=False)
    
    best_cost = np.inf
    best_index = -1
    
    for idx in indices:
        if costs[idx] < best_cost:
            best_cost = costs[idx]
            best_index = idx
            
    return population[best_index]

In [ ]:
def evolutionary_algorithm(problem, population_size, num_generations, 
                           tournament_size=3, mutation_rate=0.05, elitism=1):
    
    # === 1. Starting ===
    print(f"Start EA: Population={population_size}, Genaration={num_generations}")
    population = [starting_point(problem) for _ in range(population_size)]
    costs = [cost(problem, ind) for ind in population]
    
    best_global_sol = population[np.argmin(costs)]
    best_global_cost = np.min(costs)
    
    print(f"Geneartion 0: cost = {best_global_cost:.2f}")

    # === 2. EA ===
    for gen in range(1, num_generations + 1):
        new_population = []
        
        # Elitism
        sorted_indices = np.argsort(costs)
        for i in range(elitism):
            elite_index = sorted_indices[i]
            new_population.append(population[elite_index])
            
        # --- New genereration ---
        while len(new_population) < population_size:
            # a. Parents selection
            parent1 = tournament_selection(population, costs, k=tournament_size)
            parent2 = tournament_selection(population, costs, k=tournament_size)
            
            # b. Crossover
            offspring = order_crossover_ox1(parent1, parent2)
            
            # c. Mutation
            offspring = swap_mutation(offspring, mutation_rate=mutation_rate)
            
            new_population.append(offspring)
            
        population = new_population
        costs = [cost(problem, ind) for ind in population]
        
        # Update best Global value
        current_best_cost = np.min(costs)
        if current_best_cost < best_global_cost:
            best_global_cost = current_best_cost
            best_global_sol = population[np.argmin(costs)]
            
        if gen % 100 == 0 or gen == num_generations:
            print(f"Generation {gen}: cost = {best_global_cost:.2f}")

    print("End EA.")
    return best_global_sol, best_global_cost

In [ ]:
problem = np.load('lab2/problem_r2_1000.npy')
POP_SIZE = 100       # Popolazione (es. 50-100)
GENERATIONS = 5000   # Numero di generazioni
MUTATION_RATE = 0.1   # Probabilità di mutazione (più alta del solito per EA)
TOURNAMENT_K = 20      # Dimensione del torneo
ELITISM_COUNT = 5    # Numero di "campioni" da salvare sempre


best_ea_sol, best_ea_cost = evolutionary_algorithm(
    problem,
    population_size=POP_SIZE,
    num_generations=GENERATIONS,
    tournament_size=TOURNAMENT_K,
    mutation_rate=MUTATION_RATE,
    elitism=ELITISM_COUNT
)

print("\n--- Final Result ---")
print(f"cost: {best_ea_cost:.2f}")

# Confronto con la soluzione casuale
random_sol = starting_point(problem)
random_cost = cost(problem, random_sol)
print(f"random solution cost: {random_cost:.2f}")

Esecuzione dell'Algoritmo Evolutivo sul test_problem...
Inizio EA: Popolazione=100, Generazioni=5000
Generazione 0: Miglior costo = -1866.93
Generazione 100: Miglior costo = -20716.15
Generazione 200: Miglior costo = -28148.36
Generazione 300: Miglior costo = -31952.59
Generazione 400: Miglior costo = -34434.23
Generazione 500: Miglior costo = -36076.27
Generazione 600: Miglior costo = -37354.97
Generazione 700: Miglior costo = -38325.57
Generazione 800: Miglior costo = -39322.50
Generazione 900: Miglior costo = -39932.06
Generazione 1000: Miglior costo = -40494.23
Generazione 1100: Miglior costo = -40748.31
Generazione 1200: Miglior costo = -41069.27
Generazione 1300: Miglior costo = -41345.80
Generazione 1400: Miglior costo = -41606.82
Generazione 1500: Miglior costo = -41809.85
Generazione 1600: Miglior costo = -42013.91
Generazione 1700: Miglior costo = -42250.27
Generazione 1800: Miglior costo = -42395.49
Generazione 1900: Miglior costo = -42671.16
Generazione 2000: Miglior costo 